# Tabular Playground AUG 2021 with GPU

![](https://storage.googleapis.com/kaggle-competitions/kaggle/25225/logos/header.png)

### Please upvote if you like my notebook. Do comment If you have any queries.

<a id="1"></a>
<h2 style='background:skyblue; border:0; color:white'><center>Imports<center><h2>

In [ ]:
from sklearn.preprocessing import LabelEncoder
from functools import partial
from sklearn import ensemble
from sklearn import model_selection
from hyperopt import hp, fmin, atpe, Trials
from hyperopt.pyll.base import scope
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as ptl
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

<a id="1"></a>
<h2 style='background:skyblue; border:0; color:white'><center>Files and cross validation setup<center><h2>

In [ ]:
train= pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
train.head()

In [ ]:
train.shape

In [ ]:
train['kfold']= -1
train = train.sample(frac=1).reset_index(drop=True)
train.loc[:, "bins"] = pd.cut(
 train["loss"], bins=19, labels=False
 )
kf = model_selection.StratifiedKFold(n_splits=15)
for f, (t_, v_) in enumerate(kf.split(X=train, y=train.bins.values)):
    train.loc[v_, 'kfold'] = f
train = train.drop("bins", axis=1)

In [ ]:
X=train.iloc[:,1:].copy()
X.drop('loss',axis=1,inplace=True)

In [ ]:
target =['loss', 'kfold']
X=X
y=train[target]

<a id="1"></a>
<h2 style='background:skyblue; border:0; color:white'><center>HyperParameter Optimization<center><h2>

**Ideally Its a Classificaion problem but as the metrics that is being used is RMSE so I used Regression model.**

In [ ]:
def optimize(params,x,y):
    model= XGBRegressor(n_jobs=2,nthread=2,random_state=0,**params)
    RMSE = []
    for i in range(1,3):
        X_t=x[x.kfold != i]
        X_train=X_t.iloc[:,:-1]
        y_t=y[y.kfold !=i]
        y_train=y_t.iloc[:,0].values
        X_te=x[x.kfold == i]
        X_test=X_te.iloc[:,:-1]
        y_te=y[y.kfold==i]
        y_test=y_te.iloc[:,0].values
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        fold_ras = mean_squared_error(y_test, preds, squared=False)
        RMSE.append(fold_ras)
    print(RMSE)
    return (np.mean(RMSE))

**In versions less than 4 I have used TPE (Tree of Parzen Estimator) algorithm which is designed to optimize quantization hyperparameters to find quantization configuration that achieve an expected accuracy target and provide best possible latency improvement.**
**Adaptive TPE is recently added, adaptive TPE was invented at ElectricBrain and it is actually a series of (not so) little improvements that they experimented with on top of TPE.**


In [ ]:
param_space = {
 "learning_rate": hp.uniform("learning_rate", 0.008, 0.06),
 "max_depth": scope.int(hp.quniform("max_depth",10 , 15, 1)),
 "n_estimators": scope.int(hp.quniform("n_estimators", 1350, 1750, 1)),
    
 "tree_method": hp.choice("tree_method", ["gpu_hist"]),
 "booster": hp.choice("booster", ["gbtree"]),
 "objective": hp.choice("objective", ["reg:squarederror"]),
 "eval_metric": hp.choice("eval_metric", ["rmse"]),
 "predictor": hp.choice("predictor", ["gpu_predictor"]),
 "gpu_id": hp.choice("gpu_id", [0]),
 
 # uniform chooses a value between two values 
 "subsample": hp.uniform("subsample", 0.7, 0.98),
    "min_child_weight": hp.uniform("min_child_weight",15, 70),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.75, 0.96),
    "reg_alpha": hp.uniform("reg_alpha", 140, 200),
    "reg_lambda": hp.uniform("reg_lambda", 150, 200),
    "gamma": hp.uniform("gamma", 60, 90),
    "max_delta_step": hp.uniform("max_delta_step", 0.01, 3),
    "colsample_bylevel": hp.uniform("colsample_bylevel", 0.7, 0.96),
    "colsample_bynode": hp.uniform("colsample_bynode", 0.6, 0.96)
 }
optimization_function = partial(
 optimize,
 #param_names=param_names, 
x=X, y=y
 )   
trials = Trials() 
hopt = fmin( 
 fn=optimization_function,
 space=param_space,
 algo=atpe.suggest, 
 max_evals=15,
 trials=trials
 )
print(hopt)

### Now create a dictionary named as para and manually put max_depth and n_estimator parameter and you are good to go.

<a id="1"></a>
<h2 style='background:skyblue; border:0; color:white'><center>Training and submission<center><h2>

In [ ]:
para1={'booster': 0, 'colsample_bylevel': 0.9377629603546923, 'colsample_bynode': 0.6203129196844466, 'colsample_bytree': 0.8662523590957234, 'eval_metric': 0, 'gamma': 2.56784868937463, 'gpu_id': 0, 'learning_rate': 0.020372546225192883, 'max_delta_step': 8.174023744945664, 'max_depth': 10.0, 'min_child_weight': 342.4606309007503, 'n_estimators': 909.0, 'predictor': 0, 'reg_alpha': 4.337902688500327, 'reg_lambda': 6.925018305951919, 'subsample': 0.855931381854741, 'tree_method': 0}
para2={'booster': 0, 'colsample_bylevel': 0.7055480507601603, 'colsample_bynode': 0.7861994124766559, 'colsample_bytree': 0.8711839613327488, 'eval_metric': 0, 'gamma': 3.4191878868002514, 'gpu_id': 0, 'learning_rate': 0.011352981411403947, 'max_delta_step': 9.419376785482035, 'max_depth': 12.0, 'min_child_weight': 339.00894929804645, 'n_estimators': 1773.0, 'predictor': 0, 'reg_alpha': 4.788928093561186, 'reg_lambda': 2.811859534488748, 'subsample': 0.8972123878494193, 'tree_method': 0}
para={'booster': 0, 'colsample_bylevel': 0.7329518979370699, 'colsample_bynode': 0.8508650522327323, 'colsample_bytree': 0.7598384658099369, 'eval_metric': 0, 'gamma': 73.24552392852276, 'gpu_id': 0, 'learning_rate': 0.020917931572846417, 'max_delta_step': 2.6071993050270432, 'max_depth': 10.0, 'min_child_weight': 52.37481170728059, 'n_estimators': 1718.0, 'objective': 0, 'predictor': 0, 'reg_alpha': 166.15539016968125, 'reg_lambda': 181.6854279589871, 'subsample': 0.869912010017318, 'tree_method': 0}
para_base={'booster': 0, 'colsample_bylevel': 0.8085333759685713, 'colsample_bynode': 0.8758340404751047, 'colsample_bytree': 0.7996066384650472, 'eval_metric': 0, 'gamma': 2.756119257892028, 'gpu_id': 0, 'learning_rate': 0.01338781357269889, 'max_delta_step': 6.099261739333243, 'max_depth': 9.0, 'min_child_weight': 358.6918370093609, 'n_estimators': 1130.0, 'objective': 0, 'predictor': 0, 'reg_alpha': 2.7998966086509953, 'reg_lambda': 3.598574311888844, 'subsample': 0.968651042334799, 'tree_method': 0}

In [ ]:
model=XGBRegressor(n_jobs=4,nthread=4,random_state=0,booster='gbtree', colsample_bylevel= para['colsample_bylevel'], colsample_bynode= para['colsample_bynode'], colsample_bytree= para['colsample_bytree'], gamma= para['gamma'],eval_metric= "rmse",gpu_id=0, learning_rate=para['learning_rate'],max_delta_step=para['max_delta_step'], max_depth=10, min_child_weight=para['min_child_weight'], n_estimators=1718,predictor='gpu_predictor', reg_alpha=para['reg_alpha'],reg_lambda=para['reg_lambda'], subsample=para['subsample'], tree_method='gpu_hist')

In [ ]:
X_t=X
X_train=X_t.iloc[:,:-1]
y_t=y
y_train=y_t.iloc[:,0].values
model.fit(X_train, y_train)

In [ ]:
test1=test.iloc[:,1:]
preds= model.predict(test1)
submi= pd.DataFrame({'id':test['id'],'loss':preds})
submi.to_csv('submission1.csv',index=False)